## Load necessary modules

In [1]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

Using TensorFlow backend.
/Users/nomanshafqat/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Load RetinaNet model

In [2]:
# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases

model_path = '/Users/nomanshafqat/Documents/Github/keras-retinanet/examples/model.h5'

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
#model = models.load_model(model_path, backbone_name='resnet50', convert_model=True)

#print(model.summary())

# load label to names mapping for visualization purposes
labels_to_names = {0: 'pole'}

AnchorParameters [  1.   2.   4.   7.  12.  16.  20.]
inAnchors init [1.0, 2.0, 4.0, 7.0, 12.0, 16.0, 20.0]
inAnchors init [1.0, 2.0, 4.0, 7.0, 12.0, 16.0, 20.0]
inAnchors init [1.0, 2.0, 4.0, 7.0, 12.0, 16.0, 20.0]
inAnchors init [1.0, 2.0, 4.0, 7.0, 12.0, 16.0, 20.0]
inAnchors init [1.0, 2.0, 4.0, 7.0, 12.0, 16.0, 20.0]


/Users/nomanshafqat/anaconda/lib/python3.6/site-packages/keras/models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


## Run detection on example

In [4]:
import random

dir= "/Users/nomanshafqat/Google Drive/upwork/1knew_output/JPEGImages"

image_names=os.listdir(dir)
random.shuffle(image_names)


for image_name in image_names:
    if ".DS_S" in image_name:
        continue
    # load image
    image = read_image_bgr(os.path.join(dir,image_name+""))


    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)

    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    print("processing time: ", time.time() - start)

    # correct for image scale
    boxes /= scale

    # visualize detections
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < 0.5:
            break

        color = label_color(label)

        b = box.astype(int)
        draw_box(draw, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, b, caption)
    draw = cv2.cvtColor(draw, cv2.COLOR_RGB2BGR)
    cv2.imwrite("pred/"+image_name+".jpg",draw)


    #plt.figure(figsize=(15, 15))
    #plt.axis('off')
    #plt.imshow(draw)
    #plt.show()

processing time:  3.8882288932800293
processing time:  3.635529041290283
processing time:  4.035003185272217
processing time:  5.58424186706543


KeyboardInterrupt: 